In [1]:
import os

In [3]:
%pwd   #project working directory

'c:\\Users\\sjasm\\Documents\\Text-Summarization\\notebooks_and_research'

In [4]:
 #moving back to root dir
os.chdir("../")  #change directory

In [5]:
%pwd  #inside root

'c:\\Users\\sjasm\\Documents\\Text-Summarization'

In [30]:
#entity is just the return type of the function
#copy back to entity after running the experiment

#dataclass decorator
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [31]:
from textSummarization.constants import *
from textSummarization.utils.common import read_yaml, create_directories

#copy this to configuration manager src-> configuration.py

In [32]:
#reading config.yaml and params.yaml
#copy this to configuration manager src-> configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
#to access everything in config.yaml

In [33]:
#creating the components

#copy to scr-> components - > data_ingestion.py
import os
import urllib.request as request
import zipfile
from textSummarization.logging import logger
from textSummarization.utils.common import get_size

In [34]:
#copy to src-> components-> data_ingestion.py
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
   

In [35]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-08-31 18:29:13,098: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-31 18:29:13,104: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-31 18:29:13,108: INFO: common: created directory at: artifacts]
[2023-08-31 18:29:13,114: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-31 18:29:15,153: INFO: 355154095: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 8398995
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "5a66017e1312b8ec026951f5f49734604aa511b1fed7b4fb671425b9780a966e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 2ACC:2A998D:683B1:989BE:64F08A36
Accept-Ranges: bytes
Date: Thu, 31 Aug 2023 12:59:25 GMT
Via: 1.1 varnish
X-Served-By: cache-hyd1100022-HYD
X-Cache: HI

In [ ]:
##converting to modular coding